In [1]:
import requests
import pandas as pd
import datetime

# ==========================
# CONFIGURACIÓN
# ==========================
CLIENT_ID = "ca34ecfe814c4833abefa5c92f8e5216"
CLIENT_SECRET = "2981B2b23c374984a212C979c63a1CB5"

# Endpoint de tránsito - elegí el que quieras (ejemplo: flujo vehicular)
BASE_URL = "https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=ca34ecfe814c4833abefa5c92f8e5216&client_secret=2981B2b23c374984a212C979c63a1CB5"



In [2]:
# ==========================
# FUNCIÓN PARA OBTENER DATOS
# ==========================

def get_transito_data():
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        # Algunos endpoints permiten filtros, por ejemplo barrio, calle, etc.
        # "sentido": "NORTE"   <-- depende de la documentación
    }
    
    response = requests.get(BASE_URL, params=params)
    
    
    if response.status_code == 200:
        data = response.json()
        print("OK")
        return data
    else:
        print("Error en la petición:", response.status_code, response.text)
        return None


In [3]:
# ==========================
# GUARDAR EN CSV
# ==========================
def save_to_csv(data, filename="transito.csv"):
    # Ejemplo: los datos vienen en una lista bajo "instancias"
    instancias = data.get("instancias", [])
    
    # Aplanamos y transformamos en DataFrame
    rows = []
    for i in instancias:
        row = {
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "distancia": i.get("distancia"),
            "nombre": i.get("contenido", {}).get("nombreId"),
            "valor": i.get("contenido", {}).get("valor")
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Guardar en CSV (append si ya existe)
    try:
        old = pd.read_csv(filename)
        df = pd.concat([old, df], ignore_index=True)
    except FileNotFoundError:
        pass
    
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Datos guardados en {filename}")


In [4]:
# ==========================
# MAIN
# ==========================
if __name__ == "__main__":
    data = []
    data = get_transito_data()
    df = pd.DataFrame(data)

Error en la petición: 500 Query parameter client_secret is not repeatable
